<a href="https://colab.research.google.com/github/eduardacdantas/santander_devweek/blob/main/santader_devweek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extract
Extraindo dados de uma api - santander devweek

In [71]:
import pandas as pd

df = pd.read_csv('/content/usuarios.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[5521, 5524, 5525, 5540, 5541, 5542, 5543]


In [72]:
api_url = 'https://sdw-2023-prd.up.railway.app'

In [73]:
import requests
import json


def get_user(id):
  response = requests.get(f'{api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None


#users = [user for id in user_ids if (user := get_user(id)) is not None]

users = []
for id in user_ids:
  user = get_user(id)
  if user is not None:
    users.append(user)

print(json.dumps(users, indent=2))

[
  {
    "id": 5521,
    "name": "duda",
    "account": {
      "id": 5852,
      "number": "123458755-0",
      "agency": "001",
      "balance": 0.0,
      "limit": 1000.0
    },
    "card": {
      "id": 5362,
      "number": "1234 1234 1234 6543",
      "limit": 2250.0
    },
    "features": [],
    "news": [
      {
        "id": 10136,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Duda, invista hoje para um futuro pr\u00f3spero! Seu dinheiro pode crescer, n\u00e3o deixe parado."
      },
      {
        "id": 10137,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Duda, investir garante o seu futuro financeiro. Vamos come\u00e7ar hoje?"
      },
      {
        "id": 10154,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Duda, investir \u00e9 es

#Transform
Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.


In [74]:
!pip install cohere tiktoken openai

In [75]:
#Colocar a chave da API_KEY para rodar o projeto
openai_key = 'ADICIONE SUA CHAVE KEY'


In [76]:
import openai
openai.api_key = openai_key

def generate_ai_news(user):
  limite_final = user['card']['limit'] * 1.5
  user['card']['limit'] = limite_final

  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário e irá vender um produto."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 300 caracteres). Diga a este usuário que se ele começar a investir hoje com o nosso banco aumentaremos o limite de crétido bancário também dele em 50% em cima do limite de crédito atual dele. Mostre o limite de {limite_final} "
      }
    ]
  )

  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Olá Duda, invista com a gente hoje e aumente seu poder financeiro! Ao investir no nosso banco, seu limite de crédito aumentará em 50%, passando a ser de R$3375. Aproveite!
Olá, Tony! Imagine fazer o seu dinheiro crescer. Com os investimentos, você pode. Ao começar hoje com nosso banco, seu limite de crédito aumentará 50% - chegando a $3375! Isso é planejamento e liberdade para você.
Caro Anakin Skywalker, tornar-se um investidor é tão poderoso quanto a força! Comece a investir conosco hoje e veja seu limite de crédito saltar 50% no seu valor atual, atingindo incríveis 2250.0. Dê o salto, nós estaremos ao seu lado!
Olá Luke, invista hoje conosco e alavanque seu potencial financeiro! Ao investir, aumentaremos seu limite de crédito em 50%, ficando em $5,062.50. Veja seu dinheiro render e conquiste novos horizontes!
Caro Harry Potter, o segredo da riqueza é investir cedo. Com nosso banco, o seu dinheiro faz magia! Inicie seus investimentos hoje e aumentaremos seu limite de crédito para £33

#Load
Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [77]:
def update_user(user):
  response = requests.put(f"{api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User duda updated? True!
User tony updated? True!
User anakin skywalker updated? True!
User luke skywalker updated? True!
User Harry Potter updated? True!
User Valdemort updated? True!
User Frodo Bolseiro updated? True!
